## Joins y Merge con Data sets
+ Equivalente a las operaciones de BDs relacionales o SQL
+ Permiten relacionar datasets mediante variables existentes

In [1]:
import pandas as pd
import numpy as np

### Importe de primer Dataset

In [2]:
df_path = "../resources/python-ml-course/datasets/athletes/"

# OjO csv no esta en UTF 8 probablemente por caracteres rusos
df =  pd.read_csv(df_path + "Medals.csv", encoding="ISO-8859-1")
df.head()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals
0,Michael Phelps,23.0,2008,08/24/2008,8,0,0,8
1,Michael Phelps,19.0,2004,08/29/2004,6,0,2,8
2,Michael Phelps,27.0,2012,08/12/2012,4,2,0,6
3,Natalie Coughlin,25.0,2008,08/24/2008,1,2,3,6
4,Aleksey Nemov,24.0,2000,10/01/2000,2,1,3,6


In [18]:
df.shape

(8618, 8)

Podemos observar datos "duplicados" pero que tienen consistencia debio a que un atleta puede competir mas de un año, igualmente se analizan la cantidad de atletas para tener el dato

In [3]:
print(f"El numero total de atletas es de: ",len(df["Athlete"].unique().tolist()))

El numero total de atletas es de:  6956


In [4]:
print(f"Los atletas que han ganado medallas en mas de un año son: ",df.shape[0] - len(df["Athlete"].unique().tolist()))

Los atletas que han ganado medallas en mas de un año son:  1662


### Importe de segundo Dataset

In [3]:
df_country = pd.read_csv(df_path + "Athelete_Country_Map.csv", encoding="ISO-8859-1")
df_country.head()

,Athlete,Country
0,Michael Phelps,United States
1,Natalie Coughlin,United States
2,Aleksey Nemov,Russia
3,Alicia Coutts,Australia
4,Missy Franklin,United States


In [11]:
df_country.shape

(6970, 2)

Podemos observar que existen mas datos que en el dataset de atletas eso se debe a que ciertos paises cambiaron de nombre o se separaron durante la fecha estudiada 

### Importe tercer Dataset

In [4]:
df_sports = pd.read_csv(df_path + "Athelete_Sports_Map.csv", encoding="ISO-8859-1")
df_sports.head()

,Athlete,Sport
0,Michael Phelps,Swimming
1,Natalie Coughlin,Swimming
2,Aleksey Nemov,Gymnastics
3,Alicia Coutts,Swimming
4,Missy Franklin,Swimming


In [13]:
df_sports.shape

(6975, 2)

Nuevamente existen datos duplicados en este caso debido a atletas que se desempeñan en mas de un deporte

## Union de Datasets

### Union con Merge y columna clave

In [5]:
df_main_country = pd.merge(left=df, right=df_country,
                           left_on="Athlete", right_on="Athlete")
df_main_country.head()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
0,Michael Phelps,23.0,2008,08/24/2008,8,0,0,8,United States
1,Michael Phelps,19.0,2004,08/29/2004,6,0,2,8,United States
2,Michael Phelps,27.0,2012,08/12/2012,4,2,0,6,United States
3,Natalie Coughlin,25.0,2008,08/24/2008,1,2,3,6,United States
4,Natalie Coughlin,21.0,2004,08/29/2004,2,2,1,5,United States


In [6]:
df_main_country.shape

(8657, 9)

se puede observar que al hacer merge sin condiciones adicionales se duplicaron los datos para mantener la columna clave "Athlete" relacionada a mas de un pais, esto implica que sus medallas se duplicaron y significaria un prblema al obtener metricas personales.

In [7]:
df_main_country[df_main_country["Athlete"] == "Aleksandar Ciric"]

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
1503,Aleksandar Ciric,30.0,2008,08/24/2008,0,0,1,1,Serbia
1504,Aleksandar Ciric,30.0,2008,08/24/2008,0,0,1,1,Serbia and Montenegro
1505,Aleksandar Ciric,26.0,2004,08/29/2004,0,1,0,1,Serbia
1506,Aleksandar Ciric,26.0,2004,08/29/2004,0,1,0,1,Serbia and Montenegro
1507,Aleksandar Ciric,22.0,2000,10/01/2000,0,0,1,1,Serbia
1508,Aleksandar Ciric,22.0,2000,10/01/2000,0,0,1,1,Serbia and Montenegro


Este problema se presenta al no tener el dato de cuando el atleta participo por cada pais, ya que se genera una relacion de uno(Atleta) a muchos(Paises), la cual no se puede emplear para merge

Una solucion es eliminar los atletas duplicados en la tabla de country

In [8]:
# Fijamos la columna unica a la cual eliminar duplicados
df_country_dp = df_country.drop_duplicates(subset="Athlete")
df_country_dp.shape

(6956, 2)

Ahora con una relacion 1 Atleta - 1 pais, podemos relacionar con merge sin duplicar

In [9]:
df_main_country_dp = pd.merge(left=df, right=df_country_dp,
                           left_on="Athlete", right_on="Athlete")
df_main_country_dp.head()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
0,Michael Phelps,23.0,2008,08/24/2008,8,0,0,8,United States
1,Michael Phelps,19.0,2004,08/29/2004,6,0,2,8,United States
2,Michael Phelps,27.0,2012,08/12/2012,4,2,0,6,United States
3,Natalie Coughlin,25.0,2008,08/24/2008,1,2,3,6,United States
4,Natalie Coughlin,21.0,2004,08/29/2004,2,2,1,5,United States


In [10]:
df_main_country_dp.shape

(8618, 9)

In [11]:
df_main_country_dp.shape[0] == df.shape[0]

True

Repetimos para df_sports

In [12]:
df_sports_dp = df_sports.drop_duplicates(subset="Athlete")
df_sports_dp.shape

(6956, 2)

In [13]:
df_main_dp = pd.merge(left=df_main_country_dp, right=df_sports_dp,
                      left_on="Athlete", right_on="Athlete")

df_main_dp.head()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country,Sport
0,Michael Phelps,23.0,2008,08/24/2008,8,0,0,8,United States,Swimming
1,Michael Phelps,19.0,2004,08/29/2004,6,0,2,8,United States,Swimming
2,Michael Phelps,27.0,2012,08/12/2012,4,2,0,6,United States,Swimming
3,Natalie Coughlin,25.0,2008,08/24/2008,1,2,3,6,United States,Swimming
4,Natalie Coughlin,21.0,2004,08/29/2004,2,2,1,5,United States,Swimming


In [14]:
df_main_dp.shape

(8618, 10)

+ Podemos ver como los datos se juntaron y obtuvimos un df general a partir de 3 csv, manteniendo la cantidad de datos originales con alguna perdida de informacion

### Union con Join
Existen diferentes tipos de Joins, estos son:
+ Inner Join
+ Left Join
+ Right Join
+ Outer o Full Join

In [41]:
# generamos un df con datos faltantes
df_country_dlt = df_country_dp[:6000]
df_main_dlt = df[:6000]

#### Inner Join
+ Devuelven el "*cruce*" de los datos entre 2 dataframes
+ En terminos de conjuntos este corresponde a una interseccion
+ El numero de filas del data set resultante sera el numero de filas comunes en ambos

In [36]:
# df_main_dp contiene la info filtrada
# df_country_dlt le faltan ciertos atletas
merged_inner = pd.merge(left=df, right=df_country_dlt,
                        how="inner", left_on="Athlete", right_on="Athlete",)
merged_inner.shape

(7479, 9)

In [38]:
merged_inner.tail()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
7474,Katrin Apel,28.0,2002,02/24/2002,1,0,0,1,Germany
7475,Ferréol Cannard,27.0,2006,02/26/2006,0,0,1,1,France
7476,Sergey Chepikov,39.0,2006,02/26/2006,0,1,0,1,Russia
7477,Nikolay Kruglov,24.0,2006,02/26/2006,0,1,0,1,Russia
7478,Delphine Peretto,24.0,2006,02/26/2006,0,0,1,1,France


#### Left Join
+ Devuelve un df con las filas que posean valor en el dataset de la izq, sin importar su tienen correspondencia en la derecha.
+ Las filas que no correspondan a ninguna del df derecho tendran NaN en sus columnas
+ El numero de filas es igual al df izq
+ En terminos de conjuntos se trata del df de la izquierda mas la interseccion(columnas) del dataset de la derecha

In [39]:
merged_left = pd.merge(left=df, right=df_country_dlt,
                       how="left", left_on="Athlete", right_on="Athlete")
merged_left.shape

(8618, 9)

In [40]:
merged_left.tail()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
8613,Olena Sadovnycha,32.0,2000,10/01/2000,0,1,0,1,NaN
8614,Kateryna Serdiuk,17.0,2000,10/01/2000,0,1,0,1,NaN
8615,Wietse van Alten,21.0,2000,10/01/2000,0,0,1,1,NaN
8616,Sandra Wagner-Sachse,31.0,2000,10/01/2000,0,0,1,1,NaN
8617,Rod White,23.0,2000,10/01/2000,0,0,1,1,NaN


#### Right Join
+ Equivalente a Left pero se concentra en df derecho

In [42]:
merged_right = pd.merge(left=df_main_dlt, right=df_country_dp,
                       how="right", left_on="Athlete", right_on="Athlete")
merged_right.shape

(8101, 9)

In [43]:
merged_right.head()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
0,Michael Phelps,23.0,2008.0,08/24/2008,8.0,0.0,0.0,8.0,United States
1,Michael Phelps,19.0,2004.0,08/29/2004,6.0,0.0,2.0,8.0,United States
2,Michael Phelps,27.0,2012.0,08/12/2012,4.0,2.0,0.0,6.0,United States
3,Natalie Coughlin,25.0,2008.0,08/24/2008,1.0,2.0,3.0,6.0,United States
4,Natalie Coughlin,21.0,2004.0,08/29/2004,2.0,2.0,1.0,5.0,United States


#### Outer Join
+ Devuelve toda la informacion, en caso de no exisitir interseccion se retorna NaN.
+ El df resultante tendra la suma de las filas y la suma de columnas considerando su interseccion
+ En terminos de conjuntos es una union de conjuntos

In [56]:
df_custom = pd.concat([df_country_dlt, pd.DataFrame(
    {"Athlete": ["Sebastian"],
    "Country": ["Chile"]
    })],ignore_index=True)
df_custom.tail()

,Athlete,Country
5996,Ferréol Cannard,France
5997,Sergey Chepikov,Russia
5998,Nikolay Kruglov,Russia
5999,Delphine Peretto,France
6000,Sebastian,Chile


In [58]:
merged_outer = pd.merge(left=df, right=df_custom,
                        how="outer", left_on="Athlete", right_on="Athlete")
merged_outer.tail()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
8614,Kateryna Serdiuk,17.0,2000.0,10/01/2000,0.0,1.0,0.0,1.0,NaN
8615,Wietse van Alten,21.0,2000.0,10/01/2000,0.0,0.0,1.0,1.0,NaN
8616,Sandra Wagner-Sachse,31.0,2000.0,10/01/2000,0.0,0.0,1.0,1.0,NaN
8617,Rod White,23.0,2000.0,10/01/2000,0.0,0.0,1.0,1.0,NaN
8618,Sebastian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chile
